In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('Data/mnist', one_hot=True)

Extracting Data/mnist/train-images-idx3-ubyte.gz
Extracting Data/mnist/train-labels-idx1-ubyte.gz
Extracting Data/mnist/t10k-images-idx3-ubyte.gz
Extracting Data/mnist/t10k-labels-idx1-ubyte.gz


In [3]:
n_classes = 10
input_size = 784

x = tf.placeholder(tf.float32, shape=[None, input_size])
y = tf.placeholder(tf.float32, shape=[None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [5]:
x_image = tf.reshape(x, [-1,28,28,1])

W_conv1 = weight_variable([7, 7, 1, 100])
b_conv1 = bias_variable([100])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1) 

W_conv2 = weight_variable([4, 4, 100, 150])
b_conv2 = bias_variable([150])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_conv3 = weight_variable([4, 4, 150, 250])
b_conv3 = bias_variable([250])
h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)

W_fc1 = weight_variable([4 * 4 * 250, 300])
b_fc1 = bias_variable([300])
h_pool3_flat = tf.reshape(h_pool3, [-1, 4*4*250])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([300, n_classes])
b_fc2 = bias_variable([n_classes])
y_pred = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [6]:
diff = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_pred)
cross_entropy = tf.reduce_mean(diff)
correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
learning_rate = 0.001
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [15]:
n_steps = 25
batch_size = 32
dropout = 0.25
evaluate_every = 10
n_val_steps = mnist.test.images.shape[0] // batch_size

In [16]:
saver = tf.train.Saver(max_to_keep=5)
save_dir = 'checkpoints/'

In [21]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
best_val = 0.0
for i in range(n_steps):
    x_batch, y_batch = mnist.train.next_batch(batch_size)
    _, train_acc = sess.run([train_step, accuracy], feed_dict={x: x_batch, y: y_batch, keep_prob: dropout})
    
    if i % evaluate_every == 0:
        val_accs = []
        for j in range(n_val_steps):
            x_batch, y_batch = mnist.test.next_batch(batch_size)
            val_acc = sess.run(accuracy, feed_dict={x: x_batch, y: y_batch, keep_prob: 1.0})
            val_accs.append(val_acc)
        print('Step {:04.0f}: train_acc: {:.4f}; val_acc: {:.4f}'.format(i, train_acc, sum(val_accs)/len(val_accs)))
        if val_acc > best_val:
            saver.save(sess, save_dir+'best-model', global_step=i)
            print('Model saved')
            best_val = val_acc
saver.save(sess, save_dir+'last-model')

Step 0000: train_acc: 0.0625; val_acc: 0.1358
Model saved
Step 0010: train_acc: 0.1875; val_acc: 0.3109
Model saved
Step 0020: train_acc: 0.1250; val_acc: 0.3996


'checkpoints/last-model'

In [22]:
with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph(save_dir+'last-model.meta')
    new_saver.restore(sess, save_dir+'last-model')
    for i in range(35):
        x_batch, y_batch = mnist.train.next_batch(batch_size)
        _, train_acc = sess.run([train_step, accuracy], feed_dict={x: x_batch, y: y_batch, keep_prob: dropout})

INFO:tensorflow:Restoring parameters from checkpoints/last-model
